In [4]:
def validate(val_loader, val_loader_len, model, criterion, nb_classes, args, labels=None):
    bar = Bar('Processing', max=val_loader_len)

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    ground_truth = np.empty((0, 0))
    predicted = np.empty((0, 0))
    for i, (inputs, target) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if nb_classes > 2:
            target = target.cuda(non_blocking=True).to(dtype=torch.long)
        else:
            target = target.cuda(non_blocking=True).to(dtype=torch.float)
            target = torch.unsqueeze(target, 1)

        inputs = inputs.cuda(non_blocking=True).to(dtype=torch.float)

        with torch.no_grad():
            # compute output
            output = model(inputs)
            loss = criterion(output, target)

        # measure accuracy and record loss
        if nb_classes > 2:
            prec1 = accuracy(output, target, topk=(1,))
        else:
            prec1 = binary_acc(output, target)

        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        y_pred = output.argmax(dim=1).cpu()
        predicted = np.append(predicted, y_pred)
        ground_truth = np.append(ground_truth, target.cpu())

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix = '({batch}/{size}) Data: {data:.3f}s | Dataset: {dt:} | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f}'.format(
            batch=i + 1,
            dt=args.dataset,
            size=val_loader_len,
            data=data_time.avg,
            bt=batch_time.avg,
            total=bar.elapsed_td,
            eta=bar.eta_td,
            loss=losses.avg,
            top1=top1.avg
        )
        bar.next()
    bar.finish()

    res = None
    if labels is not None:
        ground_truth = ground_truth.astype('int')
        predicted = predicted.astype('int')
        res = classification_report(
            ground_truth, predicted, target_names=labels)

    return (losses.avg, top1.avg, res)

In [1]:
import os
import pandas as pd
import h5py
import matplotlib
import time
import shutil
import torch
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter
import numpy as np
from math import cos, pi
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from model.net import get_network
from model.config import config
from utils import Bar, Logger, AverageMeter, accuracy, mkdir_p, savefig, argparser, binary_acc
from UCR_DataLoader.UCR_Archive_Dataset import UCR_DataSet
from UCR_DataLoader.UCR_Archive_Dataset import get_data
import random
from skimage.util.shape import view_as_windows
from scipy.io import loadmat


In [2]:
def train(train_loader, train_loader_len, model, criterion, optimizer, epoch, args, nb_classes):
    bar = Bar('Processing', max=train_loader_len)

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (inputs, target) in enumerate(train_loader):
        c_lr = adjust_learning_rate(
            optimizer, epoch, i, train_loader_len, args)

        # measure data loading time
        data_time.update(time.time() - end)

        if nb_classes > 2:
            target = target.cuda(non_blocking=True).to(dtype=torch.long)
            #target = target.to(dtype=torch.long)
        else:
            target = target.cuda(non_blocking=True).to(dtype=torch.float)
            target = torch.unsqueeze(target, 1)

        inputs = inputs.cuda(non_blocking=True).to(dtype=torch.float)
        #inputs = inputs.to(dtype=torch.float)

        # compute output
        output = model(inputs)
        loss = criterion(output, target)

        # measure accuracy and record loss
        if nb_classes > 2:
            prec1 = accuracy(output, target, topk=(1,))
        else:
            prec1 = binary_acc(output, target)
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        if args.clip_grad:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix = '({batch}/{size}) | Config: {mc:} | Dataset: {dt:} | lr: {lr: .5f} | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f}'.format(
            batch=i + 1,
            mc=args.model_configuration,
            dt=args.dataset,
            size=train_loader_len,
            total=bar.elapsed_td,
            eta=bar.eta_td,
            loss=losses.avg,
            top1=top1.avg,
            lr=c_lr
        )
        bar.next()
    bar.finish()
    return (losses.avg, top1.avg)


In [3]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(
            checkpoint, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch, iteration, num_iter, args):
    lr = optimizer.param_groups[0]['lr']
    '''
    warmup_epoch = 5 if args.warmup else 0
    warmup_iter = warmup_epoch * num_iter
    current_iter = iteration + epoch * num_iter
    max_iter = args.epochs * num_iter

    if args.lr_decay == 'step':
        lr = args.lr * \
            (args.gamma ** ((current_iter - warmup_iter) // (max_iter - warmup_iter)))
    elif args.lr_decay == 'cos':
        lr = args.lr * \
            (1 + cos(pi * (current_iter - warmup_iter) / (max_iter - warmup_iter))) / 2
    elif args.lr_decay == 'linear':
        lr = args.lr * (1 - (current_iter - warmup_iter) / \
                        (max_iter - warmup_iter))
    elif args.lr_decay == 'schedule':
        count = sum([1 for s in args.schedule if s <= epoch])
        lr = args.lr * pow(args.gamma, count)
    else:
        raise ValueError('Unknown lr mode {}'.format(args.lr_decay))

    if epoch < warmup_epoch:
        lr = args.lr * current_iter / warmup_iter


    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    '''
    return lr


def convert_atcn1d_to2d(model_1d, model_2d):
    atcn1d_state = model_1d.state_dict()
    atcn2d_state = model_2d.state_dict()
    for (ent_1d_key, ent_1d_val), (ent_2d_key, ent_2d_val) in zip(atcn1d_state.items(), atcn2d_state.items()):
        assert(ent_1d_key == ent_2d_key)
        if len(ent_2d_val.shape) == 4:
            new_val = ent_2d_val.squeeze(2)
        else:
            new_val = ent_2d_val
        assert(ent_1d_val.shape == new_val.shape)
        atcn1d_state[ent_1d_key] = new_val
        pass
    model_1d.load_state_dict(atcn1d_state)


def extend_4d(x, y):

    x = x.reshape(x.shape[1], x.shape[0])
    # This function is for making a 1D input time series to 4D.
    x = torch.unsqueeze(x, 0)
    return x, y


In [10]:
import pickle

with open('args.pkl', 'rb') as f:
    args = pickle.load(f)


In [16]:
# Sampling frequency of both ppg and acceleration data in IEEE_Training dataset
fs_IEEE_Training = 125
# Sampling frequency of acceleration data in PPG_Dalia dataset
# The sampling frequency of ppg data in PPG_Dalia dataset is fs_PPG_Dalia*2
fs_PPG_Dalia = 32

fs_activity = 4

Sessioni = dict()
S = dict()
acc = dict()
ppg = dict()
activity = dict()

random.seed(20)
    
ground_truth = dict()

val = dataset
    
numbers= list(range(1,16))
session_list=random.sample(numbers,len(numbers))
for j in session_list:
    paz = j
    
    with open('/media/segate/Bits/4-1/SOP_Kamlesh_sir/data/qppg/data/' + 'PPG_FieldStudy/S' + str(j) +'/S' + str(j) +'.pkl', 'rb') as f:
        S[paz] = pickle.load(f, encoding='latin1')
    ppg[paz] = S[paz]['signal']['wrist']['BVP'][::2]
    acc[paz] = S[paz]['signal']['wrist']['ACC']
    activity[paz] = S[paz]['activity']
    ground_truth[paz] = S[paz]['label']
    
sig = dict()
act_list = []
groups= []
sig_list = []
ground_truth_list = []





with open(cf.path_PPG_Dalia+'slimmed_dalia.pkl', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

NameError: name 'view_as_windows' is not defined

In [19]:
# Loop on keys of dictionary ground_truth
for k in ground_truth:
    # Remeber to set the desired time window
    activity[k] = np.moveaxis(view_as_windows(activity[k], (4*0.5,1),4*2)[:,0,:,:],1,2)
    activity[k] = activity[k][:,:,0]
    sig[k] = np.concatenate((ppg[k],acc[k]),axis=1)
    sig[k]= np.moveaxis(view_as_windows(sig[k], (fs_PPG_Dalia*0.5,4),fs_PPG_Dalia*2)[:,0,:,:],1,2)
    groups.append(np.full(sig[k].shape[0],k))
    sig_list.append(sig[k])
    act_list.append(activity[k])
    ground_truth[k] = np.reshape(ground_truth[k], (ground_truth[k].shape[0],1))
    ground_truth_list.append(ground_truth[k])

In [20]:
#print("gruppo",groups)
groups = np.hstack(groups)
X = np.vstack(sig_list)
y = np.reshape(np.vstack(ground_truth_list),(-1,1))

act = np.vstack(act_list)

data = dict()
data['X'] = X
data['y'] = y
data['groups'] = groups
data['act'] = act

In [21]:
X.shape

(64747, 4, 16)

In [22]:
y.shape

(64697, 1)

In [36]:
with open('X2.pkl', 'wb') as f:
    pickle.dump(X, f)

with open('y2.pkl', 'wb') as f:
    pickle.dump(data['act'], f)

In [37]:
from torch.utils.data import Dataset, DataLoader

class PPGDataset(Dataset):

    def __init__(self):
        f1 = open('X2.pkl', 'rb')
        f2 = open('y2.pkl', 'rb')

        self.x = torch.from_numpy(pickle.load(f1))
        self.y = torch.from_numpy(pickle.load(f2).reshape(-1))
        f1.close()
        f2.close()
        self.n_samples = self.x.shape[0]
        self.length = self.x.shape[1]*self.x.shape[2]
        

    def __getitem__(self, index):
        tmp_x = self.x[index]
        tmp_y = self.y[index]

        tmp_x, tmp_y = extend_4d(tmp_x, tmp_y)

        return tmp_x,tmp_y

    def __len__(self):
        return self.n_samples

dataset = PPGDataset()

dataloader = DataLoader(dataset=dataset,batch_size=1024, shuffle=True, num_workers=2)

In [38]:
dataiter2 = iter(dataloader)
data2 = dataiter2.next()
feat,lab = data2

In [40]:
lab.shape

torch.Size([1024])

In [41]:
input_size = dataset.length
predict_size = 9

#input_size = train_dataset.length
#predict_size = train_dataset.num_class

model = get_network(args.model_configuration, input_size,prediction_size=predict_size, dropout=args.dropout, use_conv2d=args.conv2d)
model.cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), args.lr,
                                 weight_decay=args.weight_decay)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=np.ceil(
    args.epochs/20.).astype(int), verbose=args.verbose,  min_lr=1e-6, cooldown=np.ceil(args.epochs/40.).astype(int))

for epoch in range(args.start_epoch, args.epochs):

    print('\nEpoch: [%d | %d]' % (epoch + 1, args.epochs))

    # train for one epoch
    train_loss, train_acc = train(dataloader, len(dataloader), model, criterion, optimizer, epoch, args, predict_size)
    print(train_loss, train_acc)
    #val_loss, prec1, res = validate(dataloader, len(dataloader), model, criterion, predict_size, args=args)

    lr = optimizer.param_groups[0]['lr']

    

    #lr_scheduler.step(val_loss)


Epoch: [1 | 150]
2.041975245743915 tensor(25.3865, device='cuda:0')

Epoch: [2 | 150]
2.004748555436209 tensor(26.9047, device='cuda:0')

Epoch: [3 | 150]
2.0016321423968906 tensor(27.0623, device='cuda:0')

Epoch: [4 | 150]
1.9999967190500063 tensor(27.0669, device='cuda:0')

Epoch: [5 | 150]
1.9998559097154707 tensor(27.0823, device='cuda:0')

Epoch: [6 | 150]
1.9998172385787563 tensor(27.0808, device='cuda:0')

Epoch: [7 | 150]
1.999025362501631 tensor(27.0978, device='cuda:0')

Epoch: [8 | 150]
1.9979054185030187 tensor(27.0962, device='cuda:0')

Epoch: [9 | 150]
1.9989475297849055 tensor(27.1009, device='cuda:0')

Epoch: [10 | 150]
1.9983592970239752 tensor(27.1009, device='cuda:0')

Epoch: [11 | 150]
1.9971302115952372 tensor(27.0978, device='cuda:0')

Epoch: [12 | 150]
2.000212572161394 tensor(27.0993, device='cuda:0')

Epoch: [13 | 150]
1.9979183601832642 tensor(27.0978, device='cuda:0')

Epoch: [14 | 150]
1.9985792192342775 tensor(27.0854, device='cuda:0')

Epoch: [15 | 150]


In [42]:
torch.save(model, './model.sav')

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.